# 情報計算科学の基礎 
レポート

In [4]:
# Numpy を使うので，Numpy のライブラリーをインポート
import numpy as np

def createInitMats(n:int):
    ds = 1./(n+1)
    A = np.zeros((n,n)) #すべての成分が0のn×n行列 
    b = np.zeros(n) #すべての成分が0のサイズnのベクトル
    for i in range(n):
        A[i][i] = 2./ds
        b[i] = -ds
    for i in range(n-1):
        A[i][i+1] = -1./ds
        A[i+1][i] = -1./ds

    P = np.array( [[A[n-i-1][j] for j in range(n)] for i in range(n)] )
    q = np.array( [b[n-i-1] for i in range(n)] )

    # print(A)
    # print(P)

    # print(b)
    # print(q)

    return {'A':A,'P':P,'b':b,'q':q}

In [57]:
def gausselimination_withoutpivot(n,Aorg,borg):
  A = np.copy(Aorg)
  b = np.copy(borg)
  x = np.zeros(n)

  # 前進消去
  for j in range(0,n,1):
    for i in range(j+1,n,1):
      coe = -A[i][j]/A[j][j]
      for k in range(j,n,1):
        A[i][k]+=coe*A[j][k]
      b[i]+=coe*b[j]

  # 後退代入
  for j in range(n-1,-1,-1):
    btmp=b[j]
    for i in range(n-1,j,-1):
      btmp-=A[j][i]*x[i]
    x[j]=btmp/A[j][j]

  # 結果の出力
  return x

def gausselimination_withpivot(n,A,b):
  x = np.zeros(n)
  Aorg = np.copy(A)
  borg = np.copy(b)

  #前進消去
  for j in range(n):
    #ピポット選択
    if A[j][j] == 0:
      list = [A[i][j] for i in range(j+1,n)]
      abs_list = [abs(i) for i in list]
      m = abs_list.index(max(abs_list))
      p = m + j + 1
      #交換
      b[j],b[p] = b[p],b[j]
      for c in range(n):
        A[j][c],A[p][c] = A[p][c],A[j][c]

    for i in range(j+1,n,1):
      coe=-A[i][j]/A[j][j]
      for k in range(j,n,1):
        A[i][k]+=coe*A[j][k]
      b[i]+=coe*b[j]
    
  #後退代入・結果出力
  for j in range(n-1,-1,-1):
    btmp=b[j]
    for i in range(n-1,j,-1):
      btmp -= A[j][i]*x[i]
    x[j]=btmp/A[j][j]
  return x

In [58]:
import math
import numpy as np

# ベクトルxのL2ノルムを計算
def norm2(n, x):
    res = 0
    for i in range(n):
        res += x[i]**2
    res = math.sqrt(res)

    return res

# 行列Aとベクトルxの行列ベクトル積Axを計算
def matvec(n, A, x):
    res = np.zeros(n)
    for i in range(n):
        tmp = 0
        for j in range(n):
            tmp += A[i][j]*x[j]
        res[i] = tmp

    return res

# ベクトルxとベクトルyの内積を計算
def inner_product(n, x, y):
    res = 0
    for i in range(n):
        res += x[i]*y[i]

    return res

import numpy as np

# 共役勾配法（CG法）でAx=bを解く
def CG(n, A, b):
    # 0で初期化
    x = np.zeros(n)
    # 残差の計算
    r = b - matvec(n, A, x)
    # 残差ベクトルの設定
    p = r.copy()

    # 許容誤差epsを設定
    eps = 10**-8
    # 誤差errを計算
    err = norm2(n, r)/norm2(n, b)

    counter = 0

    # 誤差errが許容誤差epsより大きい間以下を繰り返す
    while (err > eps):
      # ここを編集する
      Ap = matvec(n,A,p)
      alpha = inner_product(n,p,r) / inner_product(n,p,Ap)
      x += alpha * p
      r -= alpha * Ap
      beta = inner_product(n,r,Ap) / inner_product(n,p,Ap)
      p = r - beta * p
    #   print(f'Ap:{Ap},\talpha:{alpha},\tx:{x},\tr:{r},\tbeta:{beta},\tp:{p}')
      err = norm2(n, r) / norm2(n, b)
      counter += 1
      # print(err)
      # if counter > 1:
      #   break
    # 解を出力
    return x

## 問1
ガウスの消去法（ピボットあり）を実装し、n = 3 の場合の Ax = b の解
x を求めよ。また、n = 3 の場合の Px = q の解 x を求め、Ax = b の解と
一致することを確認せよ。

In [59]:
n = 3
Inits = createInitMats(n)
A,P,b,q = Inits['A'], Inits['P'], Inits['b'], Inits['q']

print(f'Ax = b : x = {gausselimination_withpivot(n,A,b)}')
print(f'Px = q : x = {gausselimination_withpivot(n,P,q)}')

[[ 8. -4.  0.]
 [-4.  8. -4.]
 [ 0. -4.  8.]]
[[ 0. -4.  8.]
 [-4.  8. -4.]
 [ 8. -4.  0.]]
[-0.25 -0.25 -0.25]
[-0.25 -0.25 -0.25]
Ax = b : x = [-0.09375 -0.125   -0.09375]
Px = q : x = [-0.09375 -0.125   -0.09375]


## 問2
共役勾配法（反復解法）を実装し、n = 3 の場合の Ax = b の解 x を求
めよ。


In [60]:
print('solved by CG')
print(f'Ax = b: x = {CG(n,A,b)}')

solved by CG
Ax = b: x = [-0.09375 -0.125   -0.09375]


## 問3
n = 9 の場合の Ax = b の解 x をガウスの消去法および共役勾配法で求めよ。

In [64]:
n = 9
Inits = createInitMats(n)
A,b = Inits['A'], Inits['b']

print('solved by Gauss')
print(f'Ax = b : x = {gausselimination_withpivot(n,A,b)}')

print('solved by CG')
print(f'Ax = b : x = {CG(n,A,b)}')

[[ 20. -10.   0.   0.   0.   0.   0.   0.   0.]
 [-10.  20. -10.   0.   0.   0.   0.   0.   0.]
 [  0. -10.  20. -10.   0.   0.   0.   0.   0.]
 [  0.   0. -10.  20. -10.   0.   0.   0.   0.]
 [  0.   0.   0. -10.  20. -10.   0.   0.   0.]
 [  0.   0.   0.   0. -10.  20. -10.   0.   0.]
 [  0.   0.   0.   0.   0. -10.  20. -10.   0.]
 [  0.   0.   0.   0.   0.   0. -10.  20. -10.]
 [  0.   0.   0.   0.   0.   0.   0. -10.  20.]]
[[  0.   0.   0.   0.   0.   0.   0. -10.  20.]
 [  0.   0.   0.   0.   0.   0. -10.  20. -10.]
 [  0.   0.   0.   0.   0. -10.  20. -10.   0.]
 [  0.   0.   0.   0. -10.  20. -10.   0.   0.]
 [  0.   0.   0. -10.  20. -10.   0.   0.   0.]
 [  0.   0. -10.  20. -10.   0.   0.   0.   0.]
 [  0. -10.  20. -10.   0.   0.   0.   0.   0.]
 [-10.  20. -10.   0.   0.   0.   0.   0.   0.]
 [ 20. -10.   0.   0.   0.   0.   0.   0.   0.]]
[-0.1 -0.1 -0.1 -0.1 -0.1 -0.1 -0.1 -0.1 -0.1]
[-0.1 -0.1 -0.1 -0.1 -0.1 -0.1 -0.1 -0.1 -0.1]
solved by Gauss
Ax = b : x = [-0.045 -0.

## 第4問

行列のサイズ n が大きい場合と小さい場合で、Ax = b の解 x をガウスの消去法で求めた場合と共役勾配法で求めた場合のどちらのソルバーを使った方が演算数が少なくなるか議論せよ。A が疎行列の場合 (A の非ゼロ成分が少ない)、密行列 (A の非ゼロ成分が多い) の場合ではどうなるか。なお、共役勾配法は最大で n 反復で収束することを仮定してよい。

In [71]:
n = 2**7
Inits = createInitMats(n)
A,b = Inits['A'], Inits['b']

In [72]:
print('solved by Gauss')
print(f'Ax = b : x = \n{gausselimination_withpivot(n,A,b)}')

solved by Gauss
Ax = b : x = [-0.00384592 -0.00763175 -0.01135749 -0.01502314 -0.01862869 -0.02217415
 -0.02565952 -0.02908479 -0.03244997 -0.03575506 -0.03900006 -0.04218496
 -0.04530978 -0.0483745  -0.05137912 -0.05432366 -0.0572081  -0.06003245
 -0.06279671 -0.06550087 -0.06814494 -0.07072892 -0.07325281 -0.0757166
 -0.07812031 -0.08046391 -0.08274743 -0.08497086 -0.08713419 -0.08923743
 -0.09128057 -0.09326363 -0.09518659 -0.09704946 -0.09885223 -0.10059492
 -0.10227751 -0.10390001 -0.10546241 -0.10696473 -0.10840695 -0.10978908
 -0.11111111 -0.11237305 -0.11357491 -0.11471666 -0.11579833 -0.1168199
 -0.11778138 -0.11868277 -0.11952407 -0.12030527 -0.12102638 -0.1216874
 -0.12228832 -0.12282916 -0.1233099  -0.12373055 -0.1240911  -0.12439156
 -0.12463193 -0.12481221 -0.1249324  -0.12499249 -0.12499249 -0.1249324
 -0.12481221 -0.12463193 -0.12439156 -0.1240911  -0.12373055 -0.1233099
 -0.12282916 -0.12228832 -0.1216874  -0.12102638 -0.12030527 -0.11952407
 -0.11868277 -0.11778138 -0

In [73]:
print('solved by CG')
print(f'Ax = b : x = {CG(n,A,b)}')

solved by CG
Ax = b : x = [-1.83763835e+101  1.42305507e+103  1.42997722e+103 -6.14577754e+102
  7.43242335e+102  2.27322798e+102 -8.58256289e+102 -4.30881703e+101
  2.65797149e+101  9.97929772e+100  8.18951020e+099 -1.92067565e+100
  5.53040723e+099 -2.04364815e+099  4.33260468e+098  5.63606081e+097
 -2.93883785e+098  3.00013591e+097 -4.00749121e+097 -6.69686885e+096
 -1.49824742e+096  2.70068163e+095  3.66570248e+095 -6.76148558e+094
  1.18532392e+094  8.03397223e+093 -4.32973382e+090  1.75196050e+092
 -6.75675830e+091  1.01238354e+092 -5.59699122e+091 -3.61462425e+091
 -6.99928370e+090  4.07159346e+090 -4.73304756e+089  3.10754685e+088
 -1.48360383e+088  6.56582479e+086  5.10349660e+087 -2.27276968e+086
  8.88314640e+086 -3.63125782e+086  6.14790050e+085 -1.08853088e+085
  1.22374190e+085 -8.28422639e+083  1.30846586e+083 -2.80003331e+083
  2.19130141e+081 -9.55196708e+081  1.24834538e+081  2.77037150e+081
  9.74758169e+080  1.17790298e+080 -1.61902959e+080 -8.94987359e+078
 -3.2493

In [74]:
import time

In [78]:
n = 2**7
Inits = createInitMats(n)
A,b = Inits['A'], Inits['b']

start = time.process_time()

print('solved by Gauss')
# print(f'Ax = b : x = {gausselimination_withpivot(n,A,b)}')
gausselimination_withpivot(n,A,b)

middle = time.process_time()

print('solved by CG')
# print(f'Ax = b : x = {CG(n,A,b)}')
CG(n,A,b)

end = time.process_time()

print(f'Gauss : {middle - start}, CG : {end - middle}')


solved by Gauss
solved by CG
Gauss : 0.9785678000000075, CG : 72.01706770000004


In [75]:
n = 2**5
Inits = createInitMats(n)
A,b = Inits['A'], Inits['b']

start = time.process_time()

print('solved by Gauss')
# print(f'Ax = b : x = {gausselimination_withpivot(n,A,b)}')
gausselimination_withpivot(n,A,b)

middle = time.process_time()

print('solved by CG')
# print(f'Ax = b : x = {CG(n,A,b)}')
CG(n,A,b)

end = time.process_time()

print(f'Gauss : {middle - start}, CG : {end - middle}')


solved by Gauss
solved by CG
Gauss : 0.02212170000001379, CG : 1.559478200000001


In [76]:
n = 2**3
Inits = createInitMats(n)
A,b = Inits['A'], Inits['b']

start = time.process_time()

print('solved by Gauss')
# print(f'Ax = b : x = {gausselimination_withpivot(n,A,b)}')
gausselimination_withpivot(n,A,b)

middle = time.process_time()

print('solved by CG')
# print(f'Ax = b : x = {CG(n,A,b)}')
CG(n,A,b)

end = time.process_time()

print(f'Gauss : {middle - start}, CG : {end - middle}')


solved by Gauss
solved by CG
Gauss : 0.0012648000000012871, CG : 0.04252839999998059


In [77]:
n = 2**1
Inits = createInitMats(n)
A,b = Inits['A'], Inits['b']

start = time.process_time()

print('solved by Gauss')
# print(f'Ax = b : x = {gausselimination_withpivot(n,A,b)}')
gausselimination_withpivot(n,A,b)

middle = time.process_time()

print('solved by CG')
# print(f'Ax = b : x = {CG(n,A,b)}')
CG(n,A,b)

end = time.process_time()

print(f'Gauss : {middle - start}, CG : {end - middle}')


solved by Gauss
solved by CG
Gauss : 0.0019729999999640313, CG : 0.004365100000029543


結果、共益勾配法のほうが時間がかかるという結論に至った。